# Импорты и функции

In [1]:
import org.apache.spark.sql.{DataFrame,Dataset,Column,Row}
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.Pipeline

In [2]:
val train_df = spark.read
    .options(Map("inferSchema"->"true","header"->"true"))
    .csv("/labs/slaba05/lab05_train.csv")

val test_df = spark.read
    .options(Map("inferSchema"->"true","header"->"true"))
    .csv("/labs/slaba05/lab05_test.csv")

Waiting for a Spark session to start...

train_df = [_c0: int, ID: int ... 115 more fields]
test_df = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

In [3]:
train_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CR_PROD_CNT_IL: integer (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: double (nullable = true)
 |-- APP_REGISTR_RGN_CODE: double (nullable = true)
 |-- PRC_ACCEPTS_A_POS: double (nullable = true)
 |-- PRC_ACCEPTS_A_TK: double (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CLNT_TRUST_RELATION: string (nullable = true)
 |-- PRC_ACCEPTS_TK: double (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: double (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CNT_ACCEPTS_TK: double (nullable = true)
 

In [4]:
train_df.show(5,20,true)

-RECORD 0-------------------------------------------
 _c0                         | 333149               
 ID                          | 479990               
 CR_PROD_CNT_IL              | 0                    
 AMOUNT_RUB_CLO_PRC          | 0.0                  
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                  
 APP_REGISTR_RGN_CODE        | null                 
 PRC_ACCEPTS_A_POS           | 0.0                  
 PRC_ACCEPTS_A_TK            | 0.0                  
 TURNOVER_DYNAMIC_IL_1M      | 0.0                  
 CNT_TRAN_AUT_TENDENCY1M     | null                 
 SUM_TRAN_AUT_TENDENCY1M     | null                 
 AMOUNT_RUB_SUP_PRC          | 0.0                  
 PRC_ACCEPTS_A_AMOBILE       | 0.0                  
 SUM_TRAN_AUT_TENDENCY3M     | null                 
 CLNT_TRUST_RELATION         | null                 
 PRC_ACCEPTS_TK              | 0.0                  
 PRC_ACCEPTS_A_MTP           | 0.0                  
 REST_DYNAMIC_FDEP_1M        | 0.0            

In [5]:
// Считает количество null во всех колонках датафрейма

def countCols(columns:Array[String]):Array[Column]={
    columns.map(c=>{
      count(when(col(c).isNull,c)).alias(c)
    })
}

countCols: (columns: Array[String])Array[org.apache.spark.sql.Column]


# Предобработка данных

In [6]:
// Всего строк в датафрейме
train_df.count

320764

In [7]:
// Количество колонок в датафрейме
val col_amt = train_df.columns.length

col_amt = 117


117

In [8]:
// Смотрим кол-во null в каждой колонке
train_df.select(countCols(train_df.columns):_*).show(col_amt,20,true)

-RECORD 0-----------------------------
 _c0                         | 0      
 ID                          | 0      
 CR_PROD_CNT_IL              | 0      
 AMOUNT_RUB_CLO_PRC          | 34550  
 PRC_ACCEPTS_A_EMAIL_LINK    | 180814 
 APP_REGISTR_RGN_CODE        | 265914 
 PRC_ACCEPTS_A_POS           | 180814 
 PRC_ACCEPTS_A_TK            | 180814 
 TURNOVER_DYNAMIC_IL_1M      | 1      
 CNT_TRAN_AUT_TENDENCY1M     | 251229 
 SUM_TRAN_AUT_TENDENCY1M     | 251229 
 AMOUNT_RUB_SUP_PRC          | 34551  
 PRC_ACCEPTS_A_AMOBILE       | 180814 
 SUM_TRAN_AUT_TENDENCY3M     | 220560 
 CLNT_TRUST_RELATION         | 257829 
 PRC_ACCEPTS_TK              | 180814 
 PRC_ACCEPTS_A_MTP           | 180814 
 REST_DYNAMIC_FDEP_1M        | 1      
 CNT_TRAN_AUT_TENDENCY3M     | 220560 
 CNT_ACCEPTS_TK              | 180814 
 APP_MARITAL_STATUS          | 258989 
 REST_DYNAMIC_SAVE_3M        | 1      
 CR_PROD_CNT_VCU             | 1      
 REST_AVG_CUR                | 1      
 CNT_TRAN_MED_TENDENCY1M 

In [ ]:
// Т.к. много пропущенных значений, для фичей используем поля, где нет null

In [9]:
//Значения в поле PACK (Пакет услуг) и их кол-во.
//Почти нет null, но ничего не понятно, что они значат, поэтому не будем использовать.

train_df.groupBy($"PACK").count.orderBy($"count".desc).show()

+----+------+
|PACK| count|
+----+------+
| 102|105756|
| K01| 69478|
| O01| 45644|
| 105| 40777|
| 107| 25108|
| 103| 22430|
| 104|  6073|
| 301|  3760|
| 101|  1645|
| 109|    83|
| M01|     5|
| 108|     4|
|null|     1|
+----+------+



In [10]:
// Распределение классов в поле TARGET, есть null
train_df.groupBy($"TARGET").count.orderBy($"count".desc).show()

+------+------+
|TARGET| count|
+------+------+
|     0|294607|
|     1| 26156|
|  null|     1|
+------+------+



In [11]:
// Строка с null в поле TARGET, у которой в остальных полях также null
train_df.filter(train_df("TARGET").isNull).show(1,20,true)

-RECORD 0-----------------------------
 _c0                         | 222683 
 ID                          | 369524 
 CR_PROD_CNT_IL              | 0      
 AMOUNT_RUB_CLO_PRC          | 0.0199 
 PRC_ACCEPTS_A_EMAIL_LINK    | null   
 APP_REGISTR_RGN_CODE        | null   
 PRC_ACCEPTS_A_POS           | null   
 PRC_ACCEPTS_A_TK            | null   
 TURNOVER_DYNAMIC_IL_1M      | null   
 CNT_TRAN_AUT_TENDENCY1M     | null   
 SUM_TRAN_AUT_TENDENCY1M     | null   
 AMOUNT_RUB_SUP_PRC          | null   
 PRC_ACCEPTS_A_AMOBILE       | null   
 SUM_TRAN_AUT_TENDENCY3M     | null   
 CLNT_TRUST_RELATION         | null   
 PRC_ACCEPTS_TK              | null   
 PRC_ACCEPTS_A_MTP           | null   
 REST_DYNAMIC_FDEP_1M        | null   
 CNT_TRAN_AUT_TENDENCY3M     | null   
 CNT_ACCEPTS_TK              | null   
 APP_MARITAL_STATUS          | null   
 REST_DYNAMIC_SAVE_3M        | null   
 CR_PROD_CNT_VCU             | null   
 REST_AVG_CUR                | null   
 CNT_TRAN_MED_TENDENCY1M 

In [12]:
// Удаляем строку с null в TARGET
val train_df_clean = train_df.na.drop(Seq("TARGET"))

train_df_clean = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [13]:
train_df_clean.groupBy($"TARGET").count.orderBy($"count".desc).show()

+------+------+
|TARGET| count|
+------+------+
|     0|294607|
|     1| 26156|
+------+------+



In [14]:
// Оставляем только поля без null

val train_df_clean_cut = train_df_clean.select(
    "TARGET",
    "ID",
    "AGE",
    "CLNT_SETUP_TENOR", //Срок жизни клиента в банке
    "REST_AVG_CUR", //Средние остатки по текущим счетам
    "REST_AVG_PAYM", //Средние остатки по зарплатным счетам
    "TURNOVER_PAYM", //Средние обороты по зарплатным счетам
    "TURNOVER_CC", //Средние обороты по кредитным картам

    "CR_PROD_CNT_IL", //Кол-во открытых продуктов за отчетный период (по категориям продуктов)
    "CR_PROD_CNT_VCU",
    "CR_PROD_CNT_TOVR",
    "CR_PROD_CNT_PIL",
    "CR_PROD_CNT_CC",
    "CR_PROD_CNT_CCFP",

    "TURNOVER_DYNAMIC_IL_1M", //Тренд по среднемесячным оборотам за отчетный период (1 или 3 месяца)
    "TURNOVER_DYNAMIC_CUR_1M",
    "TURNOVER_DYNAMIC_PAYM_1M",
    "TURNOVER_DYNAMIC_CC_1M",
    "TURNOVER_DYNAMIC_IL_3M",
    "TURNOVER_DYNAMIC_CUR_3M",
    "TURNOVER_DYNAMIC_PAYM_3M",
    "TURNOVER_DYNAMIC_CC_3M",

    "REST_DYNAMIC_FDEP_1M", //Тренд среднемесяцных остатков по продуктам за отчетный период ( за 1 или 3 месяца)
    "REST_DYNAMIC_IL_1M",
    "REST_DYNAMIC_CUR_1M",
    "REST_DYNAMIC_PAYM_1M",
    "REST_DYNAMIC_CC_1M",
    "REST_DYNAMIC_SAVE_3M",
    "REST_DYNAMIC_FDEP_3M",
    "REST_DYNAMIC_PAYM_3M",
    "REST_DYNAMIC_IL_3M",
    "REST_DYNAMIC_CUR_3M",
    "REST_DYNAMIC_CC_3M",

    "LDEAL_GRACE_DAYS_PCT_MED" //Прочие продуктовые параметры за отчетный период (кредитным договорам)
)

train_df_clean_cut = [TARGET: int, ID: int ... 32 more fields]


[TARGET: int, ID: int ... 32 more fields]

In [15]:
// Проверка на null в итоговом датафрейме
val col_amt = train_df_clean_cut.columns.length
train_df_clean_cut.select(countCols(train_df_clean_cut.columns):_*).show(col_amt,20,true)

-RECORD 0-----------------------
 TARGET                   | 0   
 ID                       | 0   
 AGE                      | 0   
 CLNT_SETUP_TENOR         | 0   
 REST_AVG_CUR             | 0   
 REST_AVG_PAYM            | 0   
 TURNOVER_PAYM            | 0   
 TURNOVER_CC              | 0   
 CR_PROD_CNT_IL           | 0   
 CR_PROD_CNT_VCU          | 0   
 CR_PROD_CNT_TOVR         | 0   
 CR_PROD_CNT_PIL          | 0   
 CR_PROD_CNT_CC           | 0   
 CR_PROD_CNT_CCFP         | 0   
 TURNOVER_DYNAMIC_IL_1M   | 0   
 TURNOVER_DYNAMIC_CUR_1M  | 0   
 TURNOVER_DYNAMIC_PAYM_1M | 0   
 TURNOVER_DYNAMIC_CC_1M   | 0   
 TURNOVER_DYNAMIC_IL_3M   | 0   
 TURNOVER_DYNAMIC_CUR_3M  | 0   
 TURNOVER_DYNAMIC_PAYM_3M | 0   
 TURNOVER_DYNAMIC_CC_3M   | 0   
 REST_DYNAMIC_FDEP_1M     | 0   
 REST_DYNAMIC_IL_1M       | 0   
 REST_DYNAMIC_CUR_1M      | 0   
 REST_DYNAMIC_PAYM_1M     | 0   
 REST_DYNAMIC_CC_1M       | 0   
 REST_DYNAMIC_SAVE_3M     | 0   
 REST_DYNAMIC_FDEP_3M     | 0   
 REST_DYNA

col_amt = 34


34

In [16]:
// Статистика по выбранным полям
train_df_clean_cut.describe().show(5,20,true)

-RECORD 0----------------------------------------
 summary                  | count                
 TARGET                   | 320763               
 ID                       | 320763               
 AGE                      | 320763               
 CLNT_SETUP_TENOR         | 320763               
 REST_AVG_CUR             | 320763               
 REST_AVG_PAYM            | 320763               
 TURNOVER_PAYM            | 320763               
 TURNOVER_CC              | 320763               
 CR_PROD_CNT_IL           | 320763               
 CR_PROD_CNT_VCU          | 320763               
 CR_PROD_CNT_TOVR         | 320763               
 CR_PROD_CNT_PIL          | 320763               
 CR_PROD_CNT_CC           | 320763               
 CR_PROD_CNT_CCFP         | 320763               
 TURNOVER_DYNAMIC_IL_1M   | 320763               
 TURNOVER_DYNAMIC_CUR_1M  | 320763               
 TURNOVER_DYNAMIC_PAYM_1M | 320763               
 TURNOVER_DYNAMIC_CC_1M   | 320763               


# Модель

In [17]:
// Колонки для фичей
val cols = Array(
    "AGE",
    "CLNT_SETUP_TENOR", //Срок жизни клиента в банке
    "REST_AVG_CUR", //Средние остатки по текущим счетам
    "REST_AVG_PAYM", //Средние остатки по зарплатным счетам
    "TURNOVER_PAYM", //Средние обороты по зарплатным счетам
    "TURNOVER_CC", //Средние обороты по кредитным картам

    "CR_PROD_CNT_IL", //Кол-во открытых продуктов за отчетный период (по категориям продуктов)
    "CR_PROD_CNT_VCU",
    "CR_PROD_CNT_TOVR",
    "CR_PROD_CNT_PIL",
    "CR_PROD_CNT_CC",
    "CR_PROD_CNT_CCFP",

    "TURNOVER_DYNAMIC_IL_1M", //Тренд по среднемесячным оборотам за отчетный период (1 или 3 месяца)
    "TURNOVER_DYNAMIC_CUR_1M",
    "TURNOVER_DYNAMIC_PAYM_1M",
    "TURNOVER_DYNAMIC_CC_1M",
    "TURNOVER_DYNAMIC_IL_3M",
    "TURNOVER_DYNAMIC_CUR_3M",
    "TURNOVER_DYNAMIC_PAYM_3M",
    "TURNOVER_DYNAMIC_CC_3M",

    "REST_DYNAMIC_FDEP_1M", //Тренд среднемесяцных остатков по продуктам за отчетный период ( за 1 или 3 месяца)
    "REST_DYNAMIC_IL_1M",
    "REST_DYNAMIC_CUR_1M",
    "REST_DYNAMIC_PAYM_1M",
    "REST_DYNAMIC_CC_1M",
    "REST_DYNAMIC_SAVE_3M",
    "REST_DYNAMIC_FDEP_3M",
    "REST_DYNAMIC_PAYM_3M",
    "REST_DYNAMIC_IL_3M",
    "REST_DYNAMIC_CUR_3M",
    "REST_DYNAMIC_CC_3M",

    "LDEAL_GRACE_DAYS_PCT_MED" //Прочие продуктовые параметры за отчетный период (кредитным договорам)
)

cols = Array(AGE, CLNT_SETUP_TENOR, REST_AVG_CUR, REST_AVG_PAYM, TURNOVER_PAYM, TURNOVER_CC, CR_PROD_CNT_IL, CR_PROD_CNT_VCU, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, CR_PROD_CNT_CC, CR_PROD_CNT_CCFP, TURNOVER_DYNAMIC_IL_1M, TURNOVER_DYNAMIC_CUR_1M, TURNOVER_DYNAMIC_PAYM_1M, TURNOVER_DYNAMIC_CC_1M, TURNOVER_DYNAMIC_IL_3M, TURNOVER_DYNAMIC_CUR_3M, TURNOVER_DYNAMIC_PAYM_3M, TURNOVER_DYNAMIC_CC_3M, REST_DYNAMIC_FDEP_1M, REST_DYNAMIC_IL_1M, REST_DYNAMIC_CUR_1M, REST_DYNAMIC_PAYM_1M, REST_DYNAMIC_CC_1M, REST_DYNAMIC_SAVE_3M, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_PAYM_3M, REST_DYNAMIC_IL_3M, REST_DYNAMIC_CUR_3M, REST_DYNAMIC_CC_3M, LDEAL_GRACE_DAYS_PCT_MED)


Array(AGE, CLNT_SETUP_TENOR, REST_AVG_CUR, REST_AVG_PAYM, TURNOVER_PAYM, TURNOVER_CC, CR_PROD_CNT_IL, CR_PROD_CNT_VCU, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, CR_PROD_CNT_CC, CR_PROD_CNT_CCFP, TURNOVER_DYNAMIC_IL_1M, TURNOVER_DYNAMIC_CUR_1M, TURNOVER_DYNAMIC_PAYM_1M, TURNOVER_DYNAMIC_CC_1M, TURNOVER_DYNAMIC_IL_3M, TURNOVER_DYNAMIC_CUR_3M, TURNOVER_DYNAMIC_PAYM_3M, TURNOVER_DYNAMIC_CC_3M, REST_DYNAMIC_FDEP_1M, REST_DYNAMIC_IL_1M, REST_DYNAMIC_CUR_1M, REST_DYNAMIC_PAYM_1M, REST_DYNAMIC_CC_1M, REST_DYNAMIC_SAVE_3M, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_PAYM_3M, REST_DYNAMIC_IL_3M, REST_DYNAMIC_CUR_3M, REST_DYNAMIC_CC_3M, LDEAL_GRACE_DAYS_PCT_MED)

In [18]:
// Для создания колонки с фичами
val assembler = new VectorAssembler()
  .setInputCols(cols)
  .setOutputCol("features")

// Модель - Gradient-boosted tree regression
val gbt = new GBTRegressor()
  .setLabelCol("TARGET")
  .setFeaturesCol("features")
  .setMaxIter(10)

// Валидация на ROC AUC score
val evaluator = new BinaryClassificationEvaluator()
  .setLabelCol("TARGET")
  .setRawPredictionCol("prediction")
  .setMetricName("areaUnderROC")

val pipeline = new Pipeline()
  .setStages(Array(assembler, gbt))

assembler = vecAssembler_58b538b00805
gbt = gbtr_e5f24eb9ffe1
evaluator = binEval_c552d4f76870
pipeline = pipeline_964eb644bbf8


pipeline_964eb644bbf8

In [19]:
// Валидация модели на train и val

val Array(train_data, val_data) = train_df_clean_cut.randomSplit(Array(0.7, 0.3))

val model = pipeline.fit(train_data)
val predictions = model.transform(val_data)
val accuracy = evaluator.evaluate(predictions)

train_data = [TARGET: int, ID: int ... 32 more fields]
val_data = [TARGET: int, ID: int ... 32 more fields]
model = pipeline_964eb644bbf8
predictions = [TARGET: int, ID: int ... 34 more fields]
accuracy = 0.8027856689883565


0.8027856689883565

In [20]:
// Прогноз на тестовых данных

val model = pipeline.fit(train_df_clean_cut)
val predictions = model.transform(test_df)

model = pipeline_964eb644bbf8
predictions = [_c0: int, ID: int ... 116 more fields]


[_c0: int, ID: int ... 116 more fields]

In [21]:
// Оставляем и переименовываем нужные колонки
val pred_df = predictions
    .select("ID","prediction")
    .withColumnRenamed("ID","id")
    .withColumnRenamed("prediction","target")

pred_df = [id: int, target: double]


[id: int, target: double]

In [22]:
pred_df.show()

+------+--------------------+
|    id|              target|
+------+--------------------+
|519130| 0.10532352712065535|
|234045|0.009365383704254266|
|401256|0.004809516691642...|
|551070|0.030328511215895974|
|367285|0.008593258670558546|
|497998|0.013571868107250017|
|413082|  0.1152834086062017|
|349893| -3.1037950042329E-4|
|346337| 0.13029977308402804|
|289979|0.026000123094179504|
|510818| 0.25701450311972346|
|235935|0.013917510022612992|
|532135| 0.33517723788538045|
|564760|0.014840315340617414|
|277391| 0.08362352397578383|
|336830| 0.32687509294332917|
|356053|0.010400102297599597|
|293302| 0.04010306298802175|
|322368|0.015456357395547807|
|406041|0.004650432073084594|
+------+--------------------+
only showing top 20 rows



In [23]:
// Сохраняем csv на hdfs
pred_df.write
    .option("header",true)
    .option("delimiter","|")
    .csv("lab05.csv")

In [ ]:
spark.stop()